# Collect Central catalog end points

In [ ]:
# Import libraries 
import urllib2
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from IPython.display import display
from collections import OrderedDict
import pixiedust

print 'done'

# Prepare the output table 

In [ ]:
# %%pixie_debugger
# invoke pixie_debugger

# order is important
CATALOG_SUMMARY_MV_merge_keys=['CountyFIPS','DataCategory','DataType','HUC','OrganizationIdentifier','ReportIdentifier',
'ReportUnitIdentifier','ReportUnitName','State','WaDEURLAddress']

print CATALOG_SUMMARY_MV_merge_keys

In [ ]:
# CentralCatalog="http://www.westernstateswater.org/Idaho/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?"


States={

"ADWR":'http://www.arizonawade.com/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=ADWR', # Arizona

    
"CA_DWR":'http://wade.sdsc.edu/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=CA-DWR', # California
    
    
"CODWR":"http://www.westernstateswater.org/Colorado/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=CODWR", #COLORADO

    
"IDWR":"http://www.westernstateswater.org/Idaho/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=IDWR",  #  IDAHO 

    
"KS_KDA": "http://wade.kda.ks.gov/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=KS-KDA", # KANSAS
    

"NeDNR": "http://www.westernstateswater.org/Nebraska/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=NeDNR", #NEBRASKA
    
    
"NDWR":"http://wade.water.nv.gov/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=NDWR", #NEVADA   
   
    
"NMOSE":"http://www.westernstateswater.org/NewMexico/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=NMOSE", # NEW MEXICO
  
        
"OWRD":"http://data.owrb.ok.gov:8080/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=OWRD", #OKLAHOMA 
  

"OREGON_WRD":"http://wade.wrd.state.or.us/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=OREGON-WRD"  , #OREGON 
   

"DENR":"http://www.westernstateswater.org/SouthDakota/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=DENR", #SOUTH DAKOTA
    
       
"utwre":"http://www.water.utah.gov/DWRE/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=utwre",   # UTAH: water rights
    

"UTDWRT":"http://www.water.utah.gov/DWRT/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=UTDWRT", # UTAH: water resources

    
"TCEQ":"http://www.westernstateswater.org/TCEQ/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=TCEQ", #TEXAS
       
    
"WYWDC":"http://www.westernstateswater.org/Wyoming/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=WYWDC", #WYOMING
    
    
"WYSEO":"http://www.westernstateswater.org/WYSEO/WADE/v0.2/GetCatalog/GetCatalog_GetAll.php?orgid=WYSEO", #WYOMING

    
"WAECY":"https://fortress.wa.gov/ecy/wade/v0.2/GetCatalog/GetCatalog_GetAll?orgid=WAECY" #WASHINGTON 

}

print 'done'


# Append the XMLs and their states into one big list to loop over later

**It takes 1-2 min to finish**

In [ ]:
xml_data_all={}

for state, URL in States.items():
    print state    
    xml_data=requests.get(URL).content
    
    xml_data_all[state] = xml_data
    
print 'done'    
# print xml_data_all

# Call the Centeral Catalog

In [ ]:


Orgs=['NDWR','UTDWRT','WYWDC','CODWR','DENR','utwre','ADWR','IDWR','OREGON-WRD','UTWRE','OWRD','NMOSE','CA-DWR','WAECY','WYSEO','KS-KDA','NeDNR','TCEQ']

xml_data_all_CenteralCatalog={}

for org in Orgs:
#     if org=='OREGON_WRD':
    CenteralCatalog_URL='http://www.westernstateswater.org/CentralCatalog/WADE/v0.2/GetCatalog/GetCentralCatalog.php?orgid='+org
    print org   
    xml_data=requests.get(CenteralCatalog_URL).content

    xml_data_all_CenteralCatalog[org] = xml_data
print 'done'

# Parse the XML into Pandas data frames and save them into CSVs

In [ ]:
class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
            print parsed[key]
        if element.text:# not in {'SAMPLE1','SAMPLE2','SAMPLE3'}: # avoid sample data in the states nodes
#             print element.text
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)
import numpy as np    
big_data_frame = [] 
addition_orgs={}
Removal_orgs={}
NoChange_orgs={}
Count_CenteralCatalog_records=0
Count_web_service_records=0

for state in xml_data_all.keys():
#     print state
    for org in xml_data_all_CenteralCatalog.keys():
        if state==org:
#             print org

            xml_data = xml_data_all[state]
          
    
            xml_data_CenteralCatalog = xml_data_all_CenteralCatalog[org]

            xml2df = XML2DataFrame(xml_data)
            xml2df_CenteralCatalog = XML2DataFrame(xml_data_CenteralCatalog)

            xml_dataframe = xml2df.process_data()
            xml_dataframe.columns = xml_dataframe.columns.str.replace("{http://www.exchangenetwork.net/schema/WaDE/0.2}", '')
            x=xml_dataframe.keys()
#             df.rename(columns = {'Old Name':'New Name'})

            xml_dataframe=xml_dataframe.rename(columns = {'CountFIPS':'CountyFIPS'})

            print x
            # remove any "sample data" inside the XML 
            
            Remove=['SAMPLE1','SAMPLE2','SAMPLE3']
            xml_dataframe = xml_dataframe[~xml_dataframe['OrganizationIdentifier'].isin(Remove)]
#             display (xml_dataframe)
            
            
            (row,col)= xml_dataframe.shape
            # display (xml_dataframe)
            print state + " web-service " +" has " +str(row) + ' rows' + ' and '+  str(col) + ' columns' 

            Count_web_service_records=Count_web_service_records+row

            
            xml_CenteralCatalog = xml2df_CenteralCatalog.process_data()

            (row_CenteralCatalog,col)= xml_CenteralCatalog.shape
            Count_CenteralCatalog_records=Count_CenteralCatalog_records+row_CenteralCatalog

            
            # display (xml_dataframe)
            print org + " centeral catalog " +" has " +str(row_CenteralCatalog) + ' rows' + ' and '+  str(col) + ' columns' 
            print '---- \n \n'

            if row>row_CenteralCatalog:
                addition=row-row_CenteralCatalog
                addition_orgs[org]=addition
                
#                 print org+' has '+str(addition)+' new catalog entries'
                
                           
            elif row_CenteralCatalog<row:
                removals=row_CenteralCatalog-row
                Removal_orgs[org]=removals

#                 print org+' removed '+str(removals)+' from the the central  catalog'
                Removal_orgs[org]=removals
                

            else:
#                 print org+' has no change or updates compared to the central  catalog'
                
                NoChange_orgs[org]= 'NoChange' # this one has no value....no change or we can use 0
     
    

#             if not big_data_frame.empty:
#                 print big_data_frame
#                 big_data_frame.append(xml_dataframe)
#             else: 
#             print xml_dataframe
#             big_data_frame=pd.concat([xml_dataframe, big_data_frame])
            big_data_frame.append(xml_dataframe)
            total_frame = pd.DataFrame(big_data_frame)

            total_frame = pd.concat(big_data_frame)        



            name=org+'.csv'

            # print each csv file 
            xml_dataframe.to_csv(name, index = False)


print 'Count_web_service_records=' + str(Count_web_service_records)
print 'Count_CenteralCatalog_records='+ str(Count_CenteralCatalog_records)

print 'diffence='+ str(Count_web_service_records-Count_CenteralCatalog_records)

## print the big csv for all  
total_frame.to_csv('All.csv', index = False)
# print big_data_frame


print 'addition_orgs'
addition_orgs_Update=pd.DataFrame([addition_orgs])
display (addition_orgs_Update)

#print the group here

print 'Removal_orgs' 
Removal_orgs_Update=pd.DataFrame([Removal_orgs])
display (Removal_orgs_Update)


print 'NoChange_orgs'
Removal_org_Updatep=pd.DataFrame([NoChange_orgs])
display (Removal_org_Updatep)


print '---------------------------'
print 'done'
